## Homework: Vector Search

In this homework, we will learn more about vector search
and embedding. Like in the module, we will use Qdrant and
fastembed

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

Note: if you want to learn how vector search works under
the hood, check 
[homework 3 from 2024 cohort (questions 1-4)](../../2024/03-vector-search/homework.md)

## Embeddings

Qdrant uses [fastembed](https://github.com/qdrant/fastembed)
under the hood to turn text into vectors. 

We will now explore this library

Make sure it's installed:

```bash
pip install fastembed
```

Import it: 

```python
from fastembed import TextEmbedding
```


## Q1. Embedding the query

Embed the query: `'I just discovered the course. Can I join now?'`.
Use the `'jinaai/jina-embeddings-v2-small-en'` model. 

You should get a numpy array of size 512.

In [10]:
from fastembed import TextEmbedding

In [11]:
model = "jinaai/jina-embeddings-v2-small-en"
embed_model = TextEmbedding(model_name=model)
query_text = "I just discovered the course. Can I join now?"
embedding = next(embed_model.embed([query_text]))

In [15]:
print(f"Embedding dimension: {len(embedding)}")
print(min(embedding))

Embedding dimension: 512
-0.11726373551188797


What's the minimal value in this array?

* -0.51
* -0.11   <--
* 0
* 0.51

## Q2. Cosine similarity with another vector

Now let's embed this document:

```python
doc = 'Can I still join the course after the start date?'
```

In [20]:
import numpy as np

doc = "Can I still join the course after the start date?"
doc_embedding = next(embed_model.embed([doc]))

What's the cosine similarity between the vector for the query
and the vector for the document?

* 0.3
* 0.5
* 0.7
* 0.9   <--

In [21]:
query_vec = np.array(embedding)
doc_vec = np.array(doc_embedding)

cos_sim = np.dot(query_vec, doc_vec) / (np.linalg.norm(query_vec) * np.linalg.norm(doc_vec))
print(f"Cosine Similarity: {cos_sim:.4f}")

Cosine Similarity: 0.9009



## Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:

```python
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]
```

Compute the embeddings for the text field, and compute the 
cosine between the query vector and all the documents.

In [22]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [23]:
query_text = "I just discovered the course. Can I join now?"
embedding = next(embed_model.embed([query_text]))  # query vector

In [24]:
query_vec = np.array(embedding)

results = []

for doc in documents:
    doc_vec = next(embed_model.embed([doc['text']]))
    doc_vec = np.array(doc_vec)
    
    # Cosine similarity
    cos_sim = np.dot(query_vec, doc_vec) / (np.linalg.norm(query_vec) * np.linalg.norm(doc_vec))
    
    results.append({
        'question': doc['question'],
        'similarity': cos_sim
    })


In [25]:
sorted_results = sorted(results, key=lambda x: x['similarity'], reverse=True)

for res in sorted_results:
    print(f"Similarity: {res['similarity']:.4f} | Question: {res['question']}")

Similarity: 0.8182 | Question: Course - Can I follow the course after it finishes?
Similarity: 0.8085 | Question: Course - When will the course start?
Similarity: 0.7630 | Question: Course - Can I still join the course after the start date?
Similarity: 0.7304 | Question: How can we contribute to the course?
Similarity: 0.7133 | Question: Course - What can I do before the course starts?


In [26]:
best_index = -1
best_similarity = -1.0

for i, doc in enumerate(documents):
    doc_vec = next(embed_model.embed([doc['text']]))
    doc_vec = np.array(doc_vec)
    
    cos_sim = np.dot(query_vec, doc_vec) / (np.linalg.norm(query_vec) * np.linalg.norm(doc_vec))
    
    if cos_sim > best_similarity:
        best_similarity = cos_sim
        best_index = i

print(f"Most similar document index: {best_index}")
print(f"Cosine similarity: {best_similarity:.4f}")

Most similar document index: 1
Cosine similarity: 0.8182


What's the document index with the highest similarity? (Indexing starts from 0):

- 0
- 1 <--
- 2
- 3
- 4

Hint: if you put all the embeddings of the text field in one matrix `V` (a single 2-dimensional numpy array), then
computing the cosine becomes a matrix multiplication:

```python
V.dot(q)
```

If this hint is rather confusing you than helping, feel free
to ignore it.

## Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of
`question` and `text`:

```python
full_text = doc['question'] + ' ' + doc['text']
``` 

Embed this field and compute the cosine between it and the
query vector. What's the highest scoring document?

- 0  <--
- 1
- 2
- 3
- 4

In [27]:
best_index = -1
best_similarity = -1.0

for i, doc in enumerate(documents):
    full_text = doc['question'] + ' ' + doc['text']
    full_vec = next(embed_model.embed([full_text]))
    full_vec = np.array(full_vec)
    
    cos_sim = np.dot(query_vec, full_vec) / (np.linalg.norm(query_vec) * np.linalg.norm(full_vec))
    
    if cos_sim > best_similarity:
        best_similarity = cos_sim
        best_index = i

print(f"Most similar document index (using full_text): {best_index}")
print(f"Cosine similarity: {best_similarity:.4f}")

Most similar document index (using full_text): 0
Cosine similarity: 0.8515


Is it different from Q3? If yes, why?

The value of field `question` provide values that affect the result. 

## Q5. Selecting the embedding model

Now let's select a smaller embedding model.
What's the smallest dimensionality for models in fastembed?

- 128
- 256
- 384  <--
- 512

One of these models is `BAAI/bge-small-en`. Let's use it.

In [29]:
import json

EMBEDDING_DIMENSIONALITY = 65536

for model in TextEmbedding.list_supported_models():
    if model["dim"] < EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))
        EMBEDDING_DIMENSIONALITY = model["dim"]

{
  "model": "BAAI/bge-base-en",
  "sources": {
    "hf": "Qdrant/fast-bge-base-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.42,
  "additional_files": [],
  "dim": 768,
  "tasks": {}
}
{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}


## Q6. Indexing with qdrant (2 points)

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:

```python
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:

```python
text = doc['question'] + ' ' + doc['text']
```

After the data is inserted, use the question from Q1 for querying the collection.

In [46]:
from qdrant_client import QdrantClient, models

In [47]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [48]:
model_handle = "BAAI/bge-small-en"

In [49]:
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [50]:
# Define the collection name
collection_name = "zoomcamp-hw"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [51]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [52]:
points = []
id = 0

for course in documents_raw:
    for doc in course['documents']:

        point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc['text'], model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            } #save all needed metadata fields
        )
        points.append(point)

        id += 1

In [53]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [54]:
search(query_text)

QueryResponse(points=[ScoredPoint(id=497, version=0, score=0.86619794, payload={'text': 'Yes, you can. We will also do that next week, so don’t worry, you will learn how to do it.', 'section': '2. Machine Learning for Regression', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])

What's the highest score in the results?
(The score for the first returned record):

- 0.97
- 0.87  <--
- 0.77
- 0.67

## Submit the results

* Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2025/homework/hw2
* It's possible that your answers won't match exactly. If it's the case, select the closest one.